# Квантизация

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git@main
!pip install accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b",
    load_in_8bit=True,
    device_map='auto',
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

В этой ячейке все веса изначальной модели замораживаются

In [7]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

In [8]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

In [9]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6291456 || all params: 1322049536 || trainable%: 0.47588655558516074


Параметр r отвечает за внутренюю размерность дополнительных матриц. При r=32 обучаться в итоге будет около 3м параметров, что меньше 1 процента от всех параметров изначальное модели.

### Обучение

In [11]:
import transformers
from datasets import load_dataset

data = load_dataset("databricks/databricks-dolly-15k")
data = data.map(lambda samples: tokenizer(samples['instruction']), batched=True)
data = data.map(lambda samples: tokenizer(samples['context']), batched=True)
data = data.map(lambda samples: tokenizer(samples['response']), batched=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=50,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

In [13]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,2.743600
100,2.706300
150,2.683800
200,2.699000


TrainOutput(global_step=200, training_loss=2.7081700134277344, metrics={'train_runtime': 1075.6189, 'train_samples_per_second': 2.975, 'train_steps_per_second': 0.186, 'total_flos': 4262645403156480.0, 'train_loss': 2.7081700134277344, 'epoch': 0.21})

Сохраним модель (сохранятся только дополнительные веса)

In [14]:
model.save_pretrained('opt_1.3_lora')

Чтобы загрузить обученную модель нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [1]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "opt_1.3_lora"

model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="facebook/opt-1.3b",
                                             return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [2]:
def generate(text, tokenizer, model):
  batch = tokenizer(text, return_tensors='pt').to('cuda')
  output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.0, no_repeat_ngram_size=2)

  return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [3]:
instruction = "Explain how do you work"
generate(instruction, tokenizer, model)

"Explain how do you work with the new system?\nI'm not sure what you mean. I'm just saying that I don't think it's a good idea to have a system that rewards people for playing the game.\nYou can't play the same game for more"

In [4]:
instruction = 'Summarize text: "To test this hypothesis, Dr. Davis enlisted Tina Vu, then an undergraduate at the University of Georgia, to undertake a potentially tedious task: going through photographs of 400 monarch butterflies and measuring the amount of black and white on the edges of their wings."'
generate(instruction, tokenizer, model)

'Summarize text: "To test this hypothesis, Dr. Davis enlisted Tina Vu, then an undergraduate at the University of Georgia, to undertake a potentially tedious task: going through photographs of 400 monarch butterflies and measuring the amount of black and white on the edges of their wings."\n\nI\'m not sure I understand the point of this article.\nThe article says that the black-and-white pattern on their wing is a result of the butterfly\'s wing being folded over itself. But the article doesn\'t say that.'

In [5]:
instruction = "Tell short story"
generate(instruction, tokenizer, model)

"Tell short story, I'm a girl and I have a boyfriend.\nI'm not a guy and my girlfriend is a lesbian."

In [6]:
instruction = "Name three kinds of trees"
generate(instruction, tokenizer, model)

"Name three kinds of trees.\nI'm not sure if you're being serious or not, but I'll answer anyway.  * Pine * Maple * Oak * Ash  I'm sure there are more, I just can't think of them right now."

In [7]:
instruction = "Explain who is Gagarin"
generate(instruction, tokenizer, model)

'Explain who is Gagarin and why he is important.\nHe was the first man to orbit the Earth. He was also the only man who walked on the moon.'

In [8]:
instruction = "Give me a piece of advice"
generate(instruction, tokenizer, model)

"Give me a piece of advice.   Don't be a dick.\nI'm not a big fan of being a jerk, but I'm also not going to be an asshole. I'll try to keep that in mind."

Загрузим LoRA веса и попробуем те же промпты

In [9]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [10]:
instruction = "Explain how do you work"
generate(instruction,  tokenizer, model)

'Explain how do you work with the data?\n\nWhat is the purpose of the project? What is your goal? How do we know that you are working on the right thing? \nHow do I know you will deliver the results you promised? (What are the'

In [11]:
instruction = 'Summarize text: "To test this hypothesis, Dr. Davis enlisted Tina Vu, then an undergraduate at the University of Georgia, to undertake a potentially tedious task: going through photographs of 400 monarch butterflies and measuring the amount of black and white on the edges of their wings."'
generate(instruction,  tokenizer, model)

'Summarize text: "To test this hypothesis, Dr. Davis enlisted Tina Vu, then an undergraduate at the University of Georgia, to undertake a potentially tedious task: going through photographs of 400 monarch butterflies and measuring the amount of black and white on the edges of their wings."\n\nSummize the text in a sentence:\n"Tina Vu found that the butterflies\' wings were black on one side and had white edges on both sides. She also found the wings to be slightly larger than normal."'

In [12]:
instruction = "Name three kinds of trees"
generate(instruction,  tokenizer, model)

'Name three kinds of trees.\n\nName the three most common types of grass. \nWhat is the name of the most popular type of flower?\nWhich is more common in the United States?   What is its name? What are its common names? Which'

In [13]:
instruction = "Tell short story"
generate(instruction, tokenizer, model)

'Tell short story, poem, or essay.\n\nWrite a short poem or story. Write a poem. Read a story or poem aloud. Listen to a song. Sing a lullaby. Play a musical instrument. Draw a picture. Make a sculpture. Paint'

In [14]:
instruction = "Explain who is Gagarin"
generate(instruction, tokenizer, model)

'Explain who is Gagarin, who was the first man in space, and what he did.\nExplode the moon.'

In [15]:
instruction = "Give me a piece of advice"
generate(instruction, tokenizer, model)

"Give me a piece of advice.\n\nDon't be afraid to ask for help. You will be surprised how many people are willing to help you. If you are not sure how to do something, ask someone. It is a great way to learn. Also, don't"

Кажется, стало чуть лучше